In [3]:
import ipycanvas as ipc
import numpy as np

from time import time, sleep
from threading import Thread

In [4]:
# simulation parameters
ups = 30
dt = 1 / ups
# system parameters
mb = 1
mw = 1
Ib = 1
Iw = 1
l = 1
d = 1
g = 9.81
# initial conditions
y0 = np.array([0.1, 0])
dy0 = np.array([0, 0])

# find d2y
def calculate_dynamics(tau, y, dy):
    f = np.array([tau, -tau])
    M = np.array([[Ib + l**2 * mb, l * d * mb * np.cos(y[0])], [l * d * mb * np.cos(y[0]),   Iw + d**2 * (mb + mw)]])
    c = np.array([-l * g * mb * np.sin(y[0]), -dy[0]**2 * l * d * mb * np.sin(y[0])])
    d2y = np.linalg.inv(M).dot(f - c)
    #print(M)
    #print(c)
    return d2y

# update unicycle 
def update_unicycle(tau, y, dy):
    d2y = calculate_dynamics(tau, y, dy)
    # semi-implicit euler integration
    dy = dy + d2y * dt
    y = y + dy * dt
    return y, dy, d2y

In [50]:
update_unicycle(0, np.array([np.pi/2,0]), np.array([0,0]))

[[2.000000e+00 6.123234e-17]
 [6.123234e-17 3.000000e+00]]
[-9.81      -2.4674011]


(array([1.57624633e+00, 9.13852259e-04]),
 array([0.1635    , 0.02741557]),
 array([4.905     , 0.82246703]))

In [16]:
wheel_thickness = 3
motor_size = 5
# how many pixels one meter corresponds to, defines the scale
meter_in_pixels = 75
# convert meters to pixels 
def to_pixels(distance_in_meters):
    return distance_in_meters * meter_in_pixels

# convert radians to degrees
def to_degrees(radians):
    return radians / np.pi * 180

# convert degrees into -180 +180 degrees
def to_180degrees(degrees):
    while degrees > 180:
        degrees = degrees - 360
    while degrees < -180:
        degrees = degrees + 360
    return degrees

# draw unicycle, pos = horizontal (motor position), y = [theta, phi]
def draw_unicycle(canvas, init_pos, y, r, l, color='white'):
    # kinematic relation of the wheel with the ground assuming there is no slipping
    pos = 1 * init_pos
    pos[0] = init_pos[0] + y[1] * 2 * r
    # update canvas
    with ipc.hold_canvas(canvas):
        canvas.clear()
        canvas.translate(pos[0], pos[1])
        # 2. wheel:   
        # 2.1. rim
        canvas.line_width = wheel_thickness
        canvas.stroke_arc(0, 0, r, 0, np.pi * 2) 
        # 2.2. mark
        canvas.fill_style = 'black'
        canvas.rotate(y[1])
        canvas.fill_arc(0, r * 3 / 4, wheel_thickness / 2, 0, np.pi * 2)
        # 2.3. spokes
        canvas.line_width = wheel_thickness / 3
        canvas.begin_path()
        canvas.move_to(0, -r)
        canvas.line_to(0, r)
        canvas.move_to(-r, 0)
        canvas.line_to(r, 0)
        canvas.rotate(np.pi / 4)
        canvas.move_to(0, -r)
        canvas.line_to(0, r)
        canvas.move_to(-r, 0)
        canvas.line_to(r, 0)
        canvas.stroke()
        canvas.rotate(-np.pi / 4)
        canvas.rotate(-y[1]) # undo rotation  
        # 3. body:
        # 3.1. frame
        canvas.rotate(y[0])
        canvas.fill_style = 'blue'
        canvas.fill_rect(-1.5, 0, motor_size * 3 / 5, -l)
        # 3.2. motor
        canvas.fill_style = 'darkred'
        canvas.fill_arc(0, 0, motor_size, 0, np.pi * 2)
        # 3.3. load
        canvas.fill_style = 'blue'
        canvas.translate(0, -l)
        canvas.fill_rect(-motor_size, -motor_size, 2 * motor_size)
        canvas.translate(0, l) # undo translation
        canvas.rotate(-y[0]) # undo rotation
        canvas.translate(-pos[0], -pos[1]) # undo translation
        
# draw text
def draw_text(canvas, fps, time, y):
    pos = (5, 20)
    spacing = 14
    canvas.clear()
    canvas.font = '12px serif'
    canvas.fill_text('FPS: {:.0f}'.format(fps), pos[0], pos[1])
    canvas.fill_text('Time: {:.1f} s'.format(time), pos[0], pos[1] + spacing)
    canvas.fill_text('\u03B8: {:.1f}\u00B0'.format(to_180degrees(to_degrees(y[0]))), pos[0], pos[1] + 2 * spacing)
    canvas.fill_text('\u03C6: {:.1f}\u00B0'.format(to_180degrees(to_degrees(y[1]))), pos[0], pos[1] + 3 * spacing)

In [17]:
canvas_width = 600
canvas_height = 300
init_pos = np.array([canvas_width / 2, canvas_height * 3 / 4])
wheel_radius = to_pixels(d / 2)

# initialize canvas
multi_canvas = ipc.MultiCanvas(3, width=canvas_width, height=canvas_height)
# initialize background 
# 1. ground:
multi_canvas[0].fill_style = 'gray'
multi_canvas[0].stroke_rect(0, 0, canvas_width, canvas_height)
multi_canvas[0].fill_rect(0, init_pos[1] + wheel_radius, canvas_width, canvas_height - (init_pos[1] + wheel_radius))

#testing
draw_unicycle(multi_canvas[1], init_pos, [0,0], to_pixels(d / 2), to_pixels(l))
draw_text(multi_canvas[2], 30, 5, [0,0])

multi_canvas

MultiCanvas(height=300, width=600)

In [19]:
class Game(Thread):
    def __init__(self, canvas, y0, dy0, t0=0):
        self.y = y0
        self.dy = dy0
        self.time = t0
        self.fps = 0
        self.canvas = canvas
        self.is_running = True
        super(Game, self).__init__()

    def run(self):
        frame_time = dt
        start_time = time()
        while(self.is_running):
            tick = time()

            self.y, self.dy, _ = update_unicycle(0, self.y, self.dy)
            draw_unicycle(multi_canvas[1], init_pos, self.y, to_pixels(d / 2), to_pixels(l))
        
            self.time = tick - start_time 
            self.fps = 1 / frame_time
            draw_text(multi_canvas[2], self.fps, self.time, self.y)
            
            if dt - (time() - tick) > 0:
                sleep(dt - (time() - tick))
            frame_time = time() - tick
                
    def stop(self):
        self.is_running = False

In [20]:
game = Game(multi_canvas[1], y0, dy0)
game.start()

In [21]:
game.stop()